<a href="https://colab.research.google.com/github/guhankesav/Ml/blob/main/Decision_Tree/CSE18511_Assgn_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#CB.EN.U4CSE18511 - POML practice lab

In [2]:
import numpy as np
import pandas as pd
eps = np.finfo(float).eps # eps’ here is the smallest representable number. 
#At times we get log(0) or 0 in the denominator, to avoid that we are going to use this
from numpy import log2 as log

In [26]:
df = pd.read_csv('/content/drive/MyDrive/6th SEM/POML/golf.csv')

In [27]:
df = df.rename(columns = {"Play Golf":"PlayGolf"}) 

In [28]:
df

,Outlook,Temp,Humidity,Windy,PlayGolf
0,Rainy,Hot,High,False,No
1,Rainy,Hot,High,True,No
2,Overcast,Hot,High,False,Yes
3,Sunny,Mild,High,False,Yes
4,Sunny,Cool,Normal,False,Yes
5,Sunny,Cool,Normal,True,No
6,Overcast,Cool,Normal,True,Yes
7,Rainy,Mild,High,False,No
8,Rainy,Cool,Normal,False,Yes
9,Sunny,Mild,Normal,False,Yes


In [30]:
entropy_node = 0  #Initialize Entropy
values = df.PlayGolf.unique()  #Unique objects - 'Yes', 'No'
for value in values:
    fraction = df.PlayGolf.value_counts()[value]/len(df.PlayGolf)  
    entropy_node += -fraction*np.log2(fraction)

In [31]:
entropy_node #entropy Target 

0.9402859586706309

In [32]:
def find_entropy(df):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    entropy = 0
    values = df[Class].unique()
    for value in values:
        fraction = df[Class].value_counts()[value]/len(df[Class])
        entropy += -fraction*log(fraction)
    return entropy

def find_entropy_attribute(df,attribute):
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name
    target_variables = df[Class].unique()  #This gives all 'Yes' and 'No'
    variables = df[attribute].unique()    #This gives different features in that attribute (like 'Hot','Cold' in Temperature)
    entropy2 = 0
    for variable in variables:
        entropy_each_feature = 0
        for target_variable in target_variables:
            d = len(df[attribute][df[attribute] == variable][df.PlayGolf == target_variable])
            n = len(df[attribute][df[attribute] == variable])
            fraction = d/(n+eps)
            entropy_each_feature += -fraction*log(fraction+eps)
        fraction2 = n/len(df)
        entropy2 += -fraction2*entropy_each_feature
    return abs(entropy2)


def find_winner(df):
    Entropy_att = []
    IG = []
    for key in df.keys()[:-1]:
        Entropy_att.append(find_entropy_attribute(df,key))
        IG.append(find_entropy(df)-find_entropy_attribute(df,key))
    return df.keys()[:-1][np.argmax(IG)]

def get_subtable(df, node,value):
      return df[df[node] == value].reset_index(drop=True)


def buildTree(df,tree=None): 
    Class = df.keys()[-1]   #To make the code generic, changing target variable class name    
    #Here we build our decision tree
    #Get attribute with maximum information gain
    node = find_winner(df)
    #Get distinct value of that attribute e.g Salary is node and Low,Med and High are values
    attValue = np.unique(df[node])
    #Create an empty dictionary to create tree    
    if tree is None:                    
        tree={}
        tree[node] = {}
   #We make loop to construct a tree by calling this function recursively. 
    #In this we check if the subset is pure and stops if it is pure. 
    for value in attValue:
        st = get_subtable(df,node,value)
        cValue,cnt = np.unique(st['PlayGolf'],return_counts=True)
        if len(cnt) == 1:
            tree[node][value] = cValue[0]
        else:
            tree[node][value] = buildTree(st)
                   
    return tree

In [33]:
tree = buildTree(df)


In [34]:
import pprint
pprint.pprint(tree)

{'Outlook': {'Overcast': 'Yes',
             'Rainy': {'Humidity': {'High': 'No', 'Normal': 'Yes'}},
             'Sunny': {'Windy': {False: 'Yes', True: 'No'}}}}


In [35]:
def predict(inst,tree):
    #This function is used to predict for any input variable 
    
    #Recursively we go through the tree that we built earlier

    for nodes in tree.keys():        

        value = inst[nodes]
        tree = tree[nodes][value]
        prediction = 0

        if type(tree) is dict:
            prediction = predict(inst,tree)
        else:
            prediction = tree
            break;

    return prediction

In [55]:
data = {'Outlook':'Rainy', 'Temp':'Cool', 'Humidity':'Normal', 'Windy':'True'}
# Outlook':'Rainy', 'Temp':'Mild', 'Humidity':'High', 'Windy':'True
inst = pd.Series(data)
prediction = predict(inst, tree)
prediction

'Yes'